# FAQs





## <font size="5.5"><span style='color:#ff6666'>**Why pydap so long to download, and how can I improve it?**<font size="3">
**Answer**: There are broadly two stages at which `pydap` downloads content: `a)` during the dataset creation, and `b)` fetching numerical / array data. These look like this [continue here](Q1)...
